Importing necessary libraries

In [2]:
from tensorflow.keras.layers import Flatten,Dense,Conv2D,MaxPooling2D,Rescaling
from tensorflow.keras.models import Sequential
import pandas as pd
import numpy as np

from tensorflow.keras.utils import image_dataset_from_directory

Creating a train and a test data file using image_dataset_from_directory

In [3]:
train_path='/content/drive/MyDrive/Face Mask Dataset/Train'
test_path='/content/drive/MyDrive/Face Mask Dataset/Test'

train_data=image_dataset_from_directory(
    directory=train_path,
    image_size=(180,180),
    batch_size=128
)
test_data=image_dataset_from_directory(
    directory=test_path,
    image_size=(180,180),
    batch_size=128
)

Found 9776 files belonging to 2 classes.
Found 979 files belonging to 2 classes.


In [3]:
train_data.class_names

['WithMask', 'WithoutMask']

Training the model using CNN

In [7]:
model1=Sequential()
model1.add(Rescaling(1./255,input_shape=(180,180,3)))
model1.add(Conv2D(16,(4,4),padding='same'))
model1.add(MaxPooling2D())
model1.add(Conv2D(64,(4,4),padding='same'))
model1.add(MaxPooling2D())
model1.add(Flatten())
model1.add(Dense(128,activation='relu'))
model1.add(Dense(1,activation='sigmoid'))

model1.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

model1.fit(train_data,validation_data=test_data,epochs=5)

Epoch 1/5
51/51 [==============================] - 691s 11s/step - loss: 0.7322 - accuracy: 0.8534 - val_loss: 0.2116 - val_accuracy: 0.9438
Epoch 2/5
51/51 [==============================] - 20s 345ms/step - loss: 0.1322 - accuracy: 0.9507 - val_loss: 0.1505 - val_accuracy: 0.9551
Epoch 3/5
51/51 [==============================] - 21s 363ms/step - loss: 0.0878 - accuracy: 0.9694 - val_loss: 0.2264 - val_accuracy: 0.9091
Epoch 4/5
51/51 [==============================] - 20s 336ms/step - loss: 0.0585 - accuracy: 0.9821 - val_loss: 0.1216 - val_accuracy: 0.9714
Epoch 5/5
51/51 [==============================] - 21s 377ms/step - loss: 0.0538 - accuracy: 0.9801 - val_loss: 0.1430 - val_accuracy: 0.9663


The accuracy was good in this model 98% and test accuracy 96%

Data Augmentation

In [8]:
from tensorflow.keras.layers import RandomFlip,RandomRotation,RandomZoom


Training the model using CNN and additional layers

In [10]:
model2=Sequential()
model2.add(RandomFlip('horizontal',input_shape=(180,180,3)))
model2.add(RandomRotation(0.1))
model2.add(RandomZoom(0.1))
model2.add(Rescaling(1./255))
model2.add(Conv2D(16,(3,3),padding='same'))
model2.add(MaxPooling2D())
model2.add(Conv2D(64,(3,3),padding='same'))
model2.add(MaxPooling2D())
model2.add(Flatten())
model2.add(Dense(128,activation='relu'))
model2.add(Dense(1,activation='sigmoid'))

model2.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
model2.fit(train_data,validation_data=test_data,epochs=5)



Epoch 1/5
51/51 [==============================] - 23s 349ms/step - loss: 1.3732 - accuracy: 0.7641 - val_loss: 0.3622 - val_accuracy: 0.8764
Epoch 2/5
51/51 [==============================] - 20s 346ms/step - loss: 0.2603 - accuracy: 0.8946 - val_loss: 0.2606 - val_accuracy: 0.9122
Epoch 3/5
51/51 [==============================] - 21s 349ms/step - loss: 0.2037 - accuracy: 0.9162 - val_loss: 0.2311 - val_accuracy: 0.9203
Epoch 4/5
51/51 [==============================] - 20s 334ms/step - loss: 0.1726 - accuracy: 0.9344 - val_loss: 0.1969 - val_accuracy: 0.9305
Epoch 5/5
51/51 [==============================] - 22s 359ms/step - loss: 0.1517 - accuracy: 0.9452 - val_loss: 0.1453 - val_accuracy: 0.9530


Accuracy dropped a little after using additional layers 
accuracy=94% and test accuracy=95%

VGG16

In [4]:
from tensorflow.keras.applications import VGG16

In [5]:
vgg16=VGG16(weights='imagenet',include_top=False)
vgg16.summary()

58889256/58889256 [==============================] - 0s 0us/step
Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None, None, 3)]   0         
                                                                 
 block1_conv1 (Conv2D)       (None, None, None, 64)    1792      
                                                                 
 block1_conv2 (Conv2D)       (None, None, None, 64)    36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, None, None, 64)    0         
                                                                 
 block2_conv1 (Conv2D)       (None, None, None, 128)   73856     
                                                                 
 block2_conv2 (Conv2D)       (None, None, None, 128)   147584    
                                                              

Using VGG16 network to train our model

Keeping include_top=False because the VGG16 model has softmax function as the output layer activation function

In [6]:
model3=Sequential()
model3.add(Rescaling(1./255,input_shape=(180,180,3)))
model3.add(vgg16)
model3.add(Flatten())
model3.add(Dense(128,activation='relu'))
model3.add(Dense(1,activation='sigmoid'))
vgg16.trainable=False

model3.compile(optimizer='adam',metrics=['accuracy'],loss='binary_crossentropy')

model3.fit(train_data,validation_data=test_data,epochs=5)

Epoch 1/5
77/77 [==============================] - 1056s 12s/step - loss: 0.1635 - accuracy: 0.9438 - val_loss: 0.0337 - val_accuracy: 0.9888
Epoch 2/5
77/77 [==============================] - 33s 405ms/step - loss: 0.0204 - accuracy: 0.9931 - val_loss: 0.0245 - val_accuracy: 0.9959
Epoch 3/5
77/77 [==============================] - 35s 429ms/step - loss: 0.0102 - accuracy: 0.9970 - val_loss: 0.0184 - val_accuracy: 0.9969
Epoch 4/5
77/77 [==============================] - 36s 435ms/step - loss: 0.0068 - accuracy: 0.9985 - val_loss: 0.0198 - val_accuracy: 0.9969
Epoch 5/5
77/77 [==============================] - 36s 439ms/step - loss: 0.0051 - accuracy: 0.9989 - val_loss: 0.0144 - val_accuracy: 0.9969


We got a very good accuracy by using VGG16 network 
accuracy 99% and test accuracy =99%

So we'll be saving this model for our predictions


Saving the model

In [8]:
model3.save('cnn_model.h5')